In [1]:
# Install required packages

!pip install ultralytics -q

!pip install transformers -q

!pip install networkx -q

!pip install kaggle -q

!pip install kagglehub -q

!pip install pyyaml -q

!pip install pyvis -q

!pip install supervision -q

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from src.utils import check_data_config

# This runs the check and returns the path
DATA_YAML = check_data_config()

Config found: /workspace/dl4cv-final-project/sku110k_fixed.yaml
--------------------
names:
  0: product
path: /workspace/dl4cv-final-project/data/datasets/thedatasith/sku110k-annotations/versions/14/SKU110K_fixed
test: images/test
train: images/train
val: images/val

--------------------


In [3]:
from ultralytics import RTDETR, YOLO

# CONFIGURATION
MODEL_TYPE = "YOLO"
WEIGHTS = "yolo11l.pt"
EPOCHS = 50

# HARDWARE OPTIMIZATION (For 64-Core + RTX 5090/4090)
# - Batch: 16 (or 24 if memory allows) to saturate the GPU
# - Workers: 16 (Fast data loading without overhead)
BATCH_SIZE = 24 
WORKERS = 16    

# best hyperprameters from the tuning notebook
hyperparameters_dict = {
    'lr0': 0.01,
    'lrf': 0.01, 
    'momentum': 0.937,
    'weight_decay': 0.0005, 
    'box': 0.2, 
    'cls': 0.5,
    'hsv_h': 0.015,
    'mosaic': 1.0,
    'mixup': 0.0,
}

print(f"Starting Training for {MODEL_TYPE} on {os.environ.get('CUDA_VISIBLE_DEVICES', 'GPU')}...")

# Load Model
if MODEL_TYPE == "RTDETR":
    model = RTDETR(WEIGHTS)
else:
    model = YOLO(WEIGHTS)

# Train
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=640,
    batch=BATCH_SIZE,
    project="../runs/final_detect",
    name=f"{MODEL_TYPE.lower()}_final_run",
    patience=15,
    save=True,
    plots=True,
    
    # High-Performance Settings
    workers=WORKERS,  
    optimizer="AdamW",
    cache=True,

    # add hyper parameters from above
    **hyperparameters_dict
)

print(f"Training Complete. Best weights: ../runs/final_detect/{MODEL_TYPE.lower()}_final_run/weights/best.pt")

Starting Training for YOLO on GPU...
Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32110MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=0.2, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/dl4cv-final-project/sku110k_fixed.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_final_run, nbs=64, nms=False, opset=None, optimize=False, opti

In [5]:
print("\nRunning Final Evaluation on Test Set...")

from ultralytics import YOLO

# model = YOLO("../runs/final_detect/{MODEL_TYPE.lower()}_final_run/weights/best.pt") 
model = YOLO("../runs/final_detect/yolo_final_run/weights/best.pt") 

# Run Validation on the TEST split
# We force split='test' so it uses the 'test' folder defined in your yaml
test_metrics = model.val(data=DATA_YAML, split='test')

# Extract & Print Key Metrics
# Ultralytics stores these in the .box attribute or results_dict
print("\n" + "="*30)
print("FINAL TEST SET RESULTS")
print("="*30)
print(f"Precision:   {test_metrics.results_dict['metrics/precision(B)']:.4f}")
print(f"Recall:      {test_metrics.results_dict['metrics/recall(B)']:.4f}")
print(f"mAP@50:      {test_metrics.results_dict['metrics/mAP50(B)']:.4f}")
print(f"mAP@50-95:   {test_metrics.results_dict['metrics/mAP50-95(B)']:.4f}")
print("="*30)


Running Final Evaluation on Test Set...
Ultralytics 8.3.235 🚀 Python-3.12.3 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 5090, 32110MiB)
YOLO11l summary (fused): 190 layers, 25,280,083 parameters, 0 gradients, 86.6 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4744.3±3294.8 MB/s, size: 829.6 KB)
val: Scanning /workspace/dl4cv-final-project/data/datasets/thedatasith/sku110k-annotations/versions/14/SKU110K_fixed/labels/test.cache... 2920 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 2920/2920 510.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 183/183 3.8it/s 48.3s
                   all       2920     429411      0.917       0.88      0.939      0.619
Speed: 0.5ms preprocess, 8.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /workspace/dl4cv-final-project/notebooks/runs/detect/val4

FINAL TEST SET RESULTS
Precision:   0.9171
Recall:      0.8802
mAP@50:      0.9395
mAP@50-95: